In [1]:
from abc import ABC, abstractmethod
import csv
import glob
import os
import shutil
import xml.etree.ElementTree as ET
import random
from datetime import datetime

from absl import app
from absl import flags


In [2]:
class Job(ABC):
    """
    The Job interface declares the operations that all concrete jobs
    must implement.
    """

    @abstractmethod
    def readBenchmarkingProfiles(self) -> str:
        pass

    @abstractmethod
    def createRepository(self) -> str:
        pass

    @abstractmethod
    def copyInputFiles(self) -> str:
        pass

    @abstractmethod
    def createExecutableBatchFile(self) -> str:
        pass


"""
Concrete Jobs provide various implementations of the Job interface.
"""


class AbstractJob(Job):
    def __init__(
        self,
        BenchmarkingCSVFile_path,
        InputFiles_path,
        storage_path,
        Number_of_jobs_repetition,
    ):
        self.csvfile = BenchmarkingCSVFile_path
        self.path = InputFiles_path
        self.src = storage_path
        self.dest = Number_of_jobs_repetition

    def readBenchmarkingProfiles(self) -> list:
        job_parameters = []

        with open(self.csvfile, "r") as file:
            csv_file = csv.DictReader(file)
            for row in csv_file:
                job_parameters.append(dict(row))

        return job_parameters

    def createRepository(self) -> None:
        os.makedirs(self.path)

    def copyInputFiles(self) -> None:
        try:
            shutil.copytree(self.src, self.dest, dirs_exist_ok=True)
        except NotADirectoryError:
            shutil.copy(self.src, self.dest)

    def createExecutableBatchFile(self) -> str:
        return "{Result of the AbstractJob: createExecutableBatchFile}"


class MaxQuantJob(AbstractJob):
    def __init__(
        self,
        BenchmarkingCSVFile_path,
        InputFiles_path,
        storage_path,
        Number_of_jobs_repetition,

    ):
        super().__init__(
            BenchmarkingCSVFile_path,
            InputFiles_path,
            storage_path,
            Number_of_jobs_repetition,
        )
        # TODO: Variables needs to be initialize 
        # self.sample_files = sample_files
        # self.xml_file_path = xml_file_path
        # self.numthreads = numthreads

    def createExecutableBatchFile(self, job_parameters, path, ExecutionID) -> None:
        with open(f"{path}{job_parameters['job-name']}_batch.sh", "w+") as fb:
            fb.writelines("#!/bin/bash\n")
            fb.writelines(f"#SBATCH -p {job_parameters['partition']}\n")
            fb.writelines(f"#SBATCH --qos=regular_partitiontimelimit\n")
            fb.writelines(f"#SBATCH --job-name={job_parameters['job-name']}\n")
            fb.writelines(f"#SBATCH --ntasks=1\n")
            fb.writelines(f"#SBATCH --time={job_parameters['timelimit']}\n")
            fb.writelines(
                f"#SBATCH --cpus-per-task={job_parameters['cpus-per-task']}\n"
            )
            fb.writelines(f"#SBATCH --mem={job_parameters['mem']}G\n")
            fb.writelines(f"#SBATCH --output={path}slurm-%j.out\n")
            fb.writelines(f"#SBATCH --mail-type=ALL,ARRAY_TASKS\n")
            fb.writelines(f"#SBATCH --mail-user=romano.h@wehi.edu.au\n")

            fb.writelines(f"module load MaxQuant/2.0.2.0\n")
            fb.writelines(f"module load python/3.8.8\n")

            fb.writelines(
                f"MaxQuant {path}mqpar.xml --changeFolder {path}mqpar.post.xml {path} {path}\n"
            )

            fb.writelines(f"MaxQuant {path}mqpar.post.xml\n")

            fb.writelines(
                f"find {path} -maxdepth 1 -mindepth 1 -type f -not -regex '.*\.\(fasta\|xml\|out\|raw\|sh\)' -delete\n"
            )
            fb.writelines(
                f"find {path} -maxdepth 1 -mindepth 1 -type d -not -regex '.*\.\(d\)' -exec rm -rf '{{}}' \;\n"
            )

            fb.writelines(
                f'echo ""$SLURM_ARRAY_JOB_ID","$SLURM_ARRAY_TASK_ID"",{job_parameters["partition"]},{job_parameters["type"]},{job_parameters["job-name"]},{job_parameters["NumFiles"]},{job_parameters["cpus-per-task"]},{job_parameters["mem"]},{job_parameters["threads"]},{job_parameters["timelimit"]} >> jobs_executed_{ExecutionID}.txt\n'
            )

        os.system(f"sbatch {path}{job_parameters['job-name']}_batch.sh")

    def updateXmlFile(self, sample_files, xml_file_path, numthreads) -> None:
        tree = ET.parse(xml_file_path)
        root = tree.getroot()

        for filepath_tag in root.findall("filePaths/string"):
            root.findall("filePaths")[0].remove(filepath_tag)

        for sample_file in sample_files:
            new_path = ET.Element("string")
            new_path.text = sample_file
            root.findall("filePaths")[0].append(new_path)

        # <useDotNetCore>True</useDotNetCore>
        root.findall("useDotNetCore")[0].text = "True"
        # <numThreads>8</numThreads>
        root.findall("numThreads")[0].text = str(numthreads)

        outputfile = xml_file_path
        tree.write(outputfile)


class DiaNNJob(AbstractJob):
    def __init__(
        self,
        BenchmarkingCSVFile_path,
        InputFiles_path,
        storage_path,
        Number_of_jobs_repetition,
    ):
        super().__init__(
            BenchmarkingCSVFile_path,
            InputFiles_path,
            storage_path,
            Number_of_jobs_repetition,
        )

    def createExecutableBatchFile(
        self, job_parameters, path, sample_files, fasta_file, library_file, ExecutionID
    ) -> None:
        with open(f"{path}{job_parameters['job-name']}_batch.sh", "w+") as fb:
            fb.writelines("#!/bin/bash\n")
            fb.writelines(f"#SBATCH -p {job_parameters['partition']}\n")
            fb.writelines(f"#SBATCH --qos=regular_partitiontimelimit\n")
            fb.writelines(f"#SBATCH --job-name={job_parameters['job-name']}\n")
            fb.writelines(f"#SBATCH --ntasks=1\n")
            fb.writelines(f"#SBATCH --time={job_parameters['timelimit']}\n")
            fb.writelines(
                f"#SBATCH --cpus-per-task={job_parameters['cpus-per-task']}\n"
            )
            fb.writelines(f"#SBATCH --mem={job_parameters['mem']}G\n")
            fb.writelines(f"#SBATCH --output={path}slurm-%j.out\n")
            fb.writelines(f"#SBATCH --mail-type=ALL\n")
            fb.writelines(f"#SBATCH --mail-user=romano.h@wehi.edu.au\n")

            fb.writelines(f"module use /stornext/System/data/modulefiles/sysbio\n")
            fb.writelines(f"module load DiaNN/1.8\n")

            fb.writelines(f"mkdir {path}output\n")

            fb.writelines(f"diann-1.8 \\\n")

            for sample_file_path in sample_files:
                name_of_folder = sample_file_path.split("/")[-1]
                fb.writelines(f"--f {path}{name_of_folder} \\\n")

            if job_parameters["type"] == "lib":
                fb.writelines(f'--lib "{library_file}" \\\n')
            else:
                fb.writelines(f'--lib "" \\\n')

            fb.writelines(f"--threads 44 --verbose 1 \\\n")
            fb.writelines(f"--out {path}output/outputreport.tsv \\\n")
            fb.writelines(f"--qvalue 0.01 --matrices \\\n")
            fb.writelines(f"--out-lib {path}output/spectrallib.tsv \\\n")

            fb.writelines(f"--gen-spec-lib --predictor --fasta {fasta_file} \\\n")
            if job_parameters["type"] == "libfree":
                fb.writelines(f"--fasta-search  --min-fr-mz 200 --max-fr-mz 1800 \\\n")
                fb.writelines(
                    f"--missed-cleavages 1 --min-pep-len 7 --max-pep-len 30 --min-pr-mz 300 \\\n"
                )
                fb.writelines(
                    f"--max-pr-mz 1800 --min-pr-charge 1 --max-pr-charge 4 \\\n"
                )

            fb.writelines(f"--met-excision --cut K*,R* \\\n")
            fb.writelines(f"--mass-acc 10 --mass-acc-ms1 10.0 \\\n")
            fb.writelines(f"--use-quant \\\n")
            fb.writelines(
                f"--reanalyse --smart-profiling --peak-center --no-ifs-removal\n"
            )

            fb.writelines(
                f"find {path} -maxdepth 1 -mindepth 1 -type f -not -regex '.*\.\(fasta\|tsv\|sh\|out\)' -delete\n"
            )

            fb.writelines(
                f"find {path} -maxdepth 1 -mindepth 1 -type d -not -regex '.*\.\(d\)' -exec rm -rf '{{}}' \;\n"
            )

            fb.writelines(
                f'echo ""$SLURM_JOB_ID"",{job_parameters["partition"]},{job_parameters["type"]},{job_parameters["job-name"]},{job_parameters["NumFiles"]},{job_parameters["cpus-per-task"]},{job_parameters["mem"]},{job_parameters["threads"]},{job_parameters["timelimit"]} >> jobs_executed_{ExecutionID}.txt\n'
            )

        os.system(f"sbatch {path}{job_parameters['job-name']}_batch.sh")


In [3]:
class BenchmarkingToolCreator(ABC):
    """
    The BenchmarkingToolCreator class declares the factory method that is supposed to return an
    object of a Job class. The BenchmarkingToolCreator's subclasses usually provide the
    implementation of this method.
    """
    def __init__(self, BenchmarkingCSVFile_path, InputFiles_path, storage_path, Number_of_jobs_repetition):
        self.BenchmarkingCSVFile_path = BenchmarkingCSVFile_path
        self.InputFiles_path = InputFiles_path
        self.storage_path = storage_path
        self.Number_of_jobs_repetition = Number_of_jobs_repetition

    @abstractmethod
    def factory_method_create_job(self):
        """
        Note that the BenchmarkingToolCreator may also provide some default implementation of
        the factory method.
        """
        pass

    def runBenchmarking(self) -> str:
        """
        Also note that, despite its name, the BenchmarkingToolCreator's primary responsibility
        is not creating jobs. Usually, it contains some core business logic
        that relies on Job objects, returned by the factory method.
        Subclasses can indirectly change that business logic by overriding the
        factory method and returning a different type of job from it.
        """

        # Call the factory method to create a Job object.
        job = self.factory_method_create_job()

        # Now, use the job.
        result = f"BenchmarkingToolCreator: The same creator's code has just worked with {job.readBenchmarkingProfiles()}"

        # TODO: Inside of this method will be our core business logic
        return result


In [4]:
"""
Concrete Creators override the factory method in order to change the resulting
product's type.
"""

class MQBenchmarkingTool(BenchmarkingToolCreator):
    """
    Note that the signature of the method still uses the abstract job type,
    even though the concrete job is actually returned from the method. This
    way the BenchmarkingToolCreator can stay independent of concrete job classes.
    """
    def __init__(self,BenchmarkingCSVFile_path, InputFiles_path, storage_path, Number_of_jobs_repetition,):
        super().__init__(BenchmarkingCSVFile_path, InputFiles_path, storage_path, Number_of_jobs_repetition)

        # TODO: Variables needs to be initialize
        # Extract the list of Input filenames: .Fasta, .XML and .d
        # original_files = glob.glob(InputFiles_path + "*.d", recursive=False)

        # Create a Dictionary to store Input Files Orderly
        # MaxQuantInputFiles = {}
        # MaxQuantInputFiles["fasta_file"] = glob.glob(InputFiles_path + "*.fasta", recursive=False)[0]
        # MaxQuantInputFiles["xml_file"] = glob.glob(InputFiles_path + "*.xml", recursive=False)[0]

        # self.sample_files = sample_files
        # self.xml_file_path = xml_file_path
        # self.numthreads = numthreads
        
    def factory_method_create_job(self) -> Job:
        return MaxQuantJob(self.BenchmarkingCSVFile_path,self.InputFiles_path,self.storage_path,self.Number_of_jobs_repetition )


class DiaNNBenchmarkingTool(BenchmarkingToolCreator):
    def __init__(self,BenchmarkingCSVFile_path, InputFiles_path, storage_path, Number_of_jobs_repetition):
        super().__init__(BenchmarkingCSVFile_path, InputFiles_path, storage_path, Number_of_jobs_repetition)
        
    def factory_method_create_job(self) -> Job:
        return DiaNNJob(self.BenchmarkingCSVFile_path,self.InputFiles_path,self.storage_path,self.Number_of_jobs_repetition)


In [5]:
# TODO: Add this function to your diagram
def client_code(creator: BenchmarkingToolCreator) -> None:
    """
    The client code works with an instance of a concrete creator, albeit through
    its base interface. As long as the client keeps working with the creator via
    the base interface, you can pass it any creator's subclass.
    """

    print(
        f"Client: I'm not aware of the creator's class, but it still works.\n"
        f"{creator.runBenchmarking()}",
        end="",
    )


In [6]:
# Welcoming message
BenchTool = input("Thanks for using BenchMe! Please select '1' if you want benchmark MQ or '2' for DiaNN")

InputFiles_path = input("Absolute path of Input DiaNN input files (.d, .fasta, .tsv). I.E: '/stornext/HPCScratch/home/romano.h/Software-Projects/Local-Repositories/DiaNN/DiaNNFiles-Dataset/'")
BenchmarkingCSVFile_path = input("Name of CSV file with DiaNN Profiles for benchmarking. I.E: 'benchmarking-profiles.csv'")
storage_path = input("Absolute path of storage directory to save outputs. I.E: '/vast/scratch/users/romano.h/DiaNNBenchmarking/'")
Number_of_jobs_repetition = input("Number of times to run each benchmarking profile job. Default: 5 times")

if BenchTool == '1':
    print("App: Launched with the MQBenchmarkingTool.")
    client_code(MQBenchmarkingTool(InputFiles_path, BenchmarkingCSVFile_path, storage_path, Number_of_jobs_repetition))
elif BenchTool == '2':
    print("App: Launched with the DiaNNBenchmarkingTool.")
    client_code(DiaNNBenchmarkingTool(InputFiles_path, BenchmarkingCSVFile_path, storage_path, Number_of_jobs_repetition))
else:
    print("I am sorry, the selected option is invalid.")


App: Launched with the DiaNNBenchmarkingTool.
Client: I'm not aware of the creator's class, but it still works.
BenchmarkingToolCreator: The same creator's code has just worked with [{'partition': 'regular', 'type': 'lib', 'job-name': 'DiaNN-1', 'NumFiles': '2', 'cpus-per-task': '10', 'mem': '40', 'threads': '2', 'timelimit': '24:00:00'}, {'partition': 'regular', 'type': 'lib', 'job-name': 'DiaNN-2', 'NumFiles': '2', 'cpus-per-task': '12', 'mem': '80', 'threads': '8', 'timelimit': '24:00:00'}, {'partition': 'regular', 'type': 'lib', 'job-name': 'DiaNN-3', 'NumFiles': '2', 'cpus-per-task': '14', 'mem': '120', 'threads': '16', 'timelimit': '24:00:00'}, {'partition': 'regular', 'type': 'lib', 'job-name': 'DiaNN-4', 'NumFiles': '2', 'cpus-per-task': '16', 'mem': '160', 'threads': '32', 'timelimit': '24:00:00'}, {'partition': 'regular', 'type': 'lib', 'job-name': 'DiaNN-5', 'NumFiles': '2', 'cpus-per-task': '18', 'mem': '200', 'threads': '64', 'timelimit': '24:00:00'}, {'partition': 'regula

In [ ]:
FLAGS = flags.FLAGS
flags.DEFINE_string("files", None, "Absolute path of Input DiaNN input files (.d, .fasta, .tsv). I.E: '/stornext/HPCScratch/home/romano.h/Software-Projects/Local-Repositories/DiaNN/DiaNNFiles-Dataset/'")
flags.DEFINE_string("profiles", None, "Name of CSV file with DiaNN Profiles for benchmarking. I.E: 'benchmarking-profiles.csv'")
flags.DEFINE_string("storage", None, "Absolute path of storage directory to save outputs. I.E: '/vast/scratch/users/romano.h/DiaNNBenchmarking/'")
flags.DEFINE_integer("repeat", 5,"Number of times to run each benchmarking profile job. Default: 5 times")

# Required flag.
flags.mark_flag_as_required("files")
flags.mark_flag_as_required("profiles")
flags.mark_flag_as_required("storage")

In [ ]:
if __name__ == "__main__":
    print("App: Launched with the MQBenchmarkingTool.")
    client_code(MQBenchmarkingTool())
    print("\n")

    print("App: Launched with the DiaNNBenchmarkingTool.")
    client_code(DiaNNBenchmarkingTool())
